In [ ]:
ls -lh /content/sample_data

In [ ]:
import pandas as pd

quartet = pd.read_json('/content/sample_data/anscombe.json')

In [ ]:
quartet.sample(5)

In [ ]:
data = quartet[quartet.Series=='I'].set_index('X').sort_index()[['Y']]
data.plot(style='+')

Регре́ссия (лат. regressio — обратное движение, отход) в теории вероятностей и математической статистике — односторонняя стохастическая зависимость, устанавливающая соответствие между случайными переменными, то есть математическое выражение, отражающее связь между зависимой переменной у и независимыми переменными х при условии, что это выражение будет иметь статистическую значимость.

Если мы точно знаем как устроено распределение x и y - решением задачи занимается математика. А если у нас есть только выборка?

In [ ]:
a = 5
def foo(x):
  return a * np.sqrt(x)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xx = np.linspace(4, 14)
plt.plot(xx, '+')
xx

In [ ]:
data.plot(style='+')
plt.plot(xx, foo(xx))
plt.show()

## Интерполяция
В каком-то смысле задача похожая - провести линию/поверхность через точки. Заполнить пространство между ними какими-то промежуточными значениями.

In [ ]:
from scipy import interpolate
interp = interpolate.interp1d(data.index, data.Y, kind='nearest')

data.plot(style='+')
xx = np.linspace(data.index.min(), data.index.max(), 100)
plt.plot(xx, interp(xx))
plt.show()

In [ ]:
data.plot(style='+')
plt.plot(xx, interpolate.interp1d(data.index, data.Y, kind='quadratic')(xx))
plt.show()

In [ ]:
data.plot(style='+')
plt.plot(xx, interpolate.interp1d(data.index, data.Y, kind='nearest')(xx))
plt.show()

# Регрессии

## Линейная (метод наименьших квадратов)

Линейные методы предполагают, что между признаками объекта (features) и целевой переменной (target/label) существует линейная зависимость, то есть
$$y = w_1 x_1 + w_2 x_2 + ... + w_k x_k + b, $$ где   
$у$ --- целевая переменная (что мы хотим предсказать),   
$x_i$ --- признак объекта $х$,   
$w_i$ --- вес $i$-го признака,   
$b$ --- bias (смещение, свободный член)  

  
**Функция потерь** --- это мера количества ошибок, которые наша линейная регрессия делает на наборе данных 


$$
\begin{aligned}
L(y_{pred}, Y) &=  \frac{1}{n}\sum_{i=1}^{n}\left(y_{pred} - Y\right)^2 
\end{aligned}
$$

In [ ]:
foo = lambda x: 2 * x/3 + 2
data.plot(style='+')
plt.plot(xx, foo(xx))
plt.show()

In [ ]:
X = data.index.values.reshape(-1, 1)
xx = np.linspace(data.index.min(), data.index.max(), 100).reshape(-1, 1)
print(X.shape, xx.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X, data.Y)
print(linear.coef_, linear.intercept_)
a = linear.coef_[0]
b = linear.intercept_
data.plot(style='+')
plt.plot(xx, a * xx + b)
plt.show()

In [ ]:
data.plot(style='+')
plt.plot(xx, linear.predict(xx))
plt.show()

## Полиномиальная

Вместо прямых линий подбираем параболы, гиперболы и т.д.

Конкретно в sklearn для этого применяют не отдельную модель, а препроцессор который добавляет степени x к набору входных параметров.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)
x2 = poly.fit_transform(X)
plt.plot(x2)
x2

In [ ]:
model2 = LinearRegression(fit_intercept=False).fit(x2, data.Y)
model2.coef_

In [ ]:
data.plot(style='+')
xx2 = poly.transform(xx.reshape(-1, 1))
plt.plot(xx, model2.predict(xx2))
plt.show()

В `Pipeline` хранятся все этапы рабочего процесса в виде единого объекта

In [ ]:
from sklearn.pipeline import Pipeline

model = Pipeline([('poly', PolynomialFeatures(degree=12)),
                  ('linear', LinearRegression(fit_intercept=False))])

model.fit(X, data.Y)
data.plot(style='+')
plt.plot(xx, model.predict(xx))
model['linear'].coef_

Видим возможное переобучение, но компьютер не умеет смотреть на графики. Как формализовать этот вопрос?

## Валидация

In [ ]:
from sklearn.metrics import mean_squared_error
score=[]
X_ = data.index.values.reshape(-1, 1)
for n in range(12):
  model = Pipeline([('poly', PolynomialFeatures(degree=n)),
                    ('linear', LinearRegression(fit_intercept=False))])
  model.fit(X_, data['Y'])
  score.append(mean_squared_error(data['Y'], model.predict(X_)))
plt.plot(score)
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
Xtr, Xtest, Ytr, Ytest = train_test_split(data.index.values.reshape(-1, 1), data['Y'], random_state=11)
print([d.shape for d in (Xtr, Xtest, Ytr, Ytest)])

In [ ]:
train_score=[]
test_score=[]
for n in range(5):
  model = Pipeline([('poly', PolynomialFeatures(degree=n)),
                    ('linear', LinearRegression(fit_intercept=False))])
  model.fit(Xtr, Ytr)
  train_score.append(mean_squared_error(Ytr, model.predict(Xtr)))
  test_score.append(mean_squared_error(Ytest, model.predict(Xtest)))

In [ ]:
score = pd.DataFrame({'train': train_score, 'test': test_score})
score.train.plot()
plt.show()

In [ ]:
score.test.plot()
plt.show()

score.test.plot(logy=True)
plt.show()

In [ ]:
plt.plot(Xtr, Ytr, '+')
plt.plot(Xtest, Ytest, '+')
plt.plot(xx, Pipeline([('poly', PolynomialFeatures(degree=1)),
                  ('linear', LinearRegression(fit_intercept=False))]).fit(Xtr, Ytr).predict(xx))
plt.plot(xx, Pipeline([('poly', PolynomialFeatures(degree=4)),
                  ('linear', LinearRegression(fit_intercept=False))]).fit(Xtr, Ytr).predict(xx))
plt.show()

## Кроссвалидация

<img src='https://drive.google.com/uc?id=19TOWCsLwIjNSmcHzu46f6JqY5TEefm9h' width=600/>



In [ ]:
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(32, test_size=2, random_state=13)
cv

In [ ]:
from sklearn.model_selection import cross_val_score
score = pd.DataFrame(columns=['mean', 'std'])
for n in range(14):
  model = Pipeline([('poly', PolynomialFeatures(degree=n)),
                    ('linear', LinearRegression(fit_intercept=False))])
  sc = cross_val_score(model, X, data.Y, cv=cv, scoring='neg_mean_squared_error')
  score.loc[n] = [-sc.mean(), sc.std()]
score

In [ ]:
score.plot(logy=True)
plt.show()

In [ ]:
score[:4]['mean'].plot()
score['mean'].min()

In [ ]:
plt.plot(X, data.Y, '+')
plt.plot(xx, Pipeline([('poly', PolynomialFeatures(degree=1)),
                  ('linear', LinearRegression(fit_intercept=False))]).fit(X, data.Y).predict(xx))
plt.plot(xx, Pipeline([('poly', PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))]).fit(X, data.Y).predict(xx))

plt.show()

## Модели с регуляризацией

Борьба с переобучением со стороны уменьшения пространства поиска в окрестности небольших значений параметров модели.

Суть регуляризации состоит в том, чтобы добавлять к функции потерь слагаемое, ограничивающее рост весов модели.   
Например, обычная версия линейной регрессии выглядит так:
$$\frac{\sum\limits_{i=1}^{\ell}\left|\left|\langle x^i, w\rangle - y^i\right|\right|^2}{\ell} \rightarrow \min_{w}.$$

Регуляризованная версия:
$$\frac{\sum\limits_{i=1}^{\ell}\left|\left|\langle x^i, w\rangle - y^i\right|\right|^2}{\ell} + \lambda\left|\left|w\right|\right|_2^2\rightarrow \min_{w}.$$

Такая версия линейной регресси называется **Ridge**-регрессией.  


В **LASSO** мы штрафуем модель  **на сумму модулей всех ее весов** (на l1-норму весов), таким образом:

$$\frac{\sum\limits_{i=1}^{\ell}\left|\left|\langle x^i, w\rangle - y^i\right|\right|^2}{\ell} + \lambda\left|\left|w\right|\right|_1\rightarrow \min_{w}.$$

**ElasticNet** использует как L1, так и L2 регуляризации:

$$\frac{\sum\limits_{i=1}^{\ell}\left|\left|\langle x^i, w\rangle - y^i\right|\right|^2}{\ell}  + \lambda_2\left|\left|w\right|\right|_2^2 + \lambda_1\left|\left|w\right|\right|_1 \rightarrow \min_{w}.$$


In [ ]:
model_poly11 = Pipeline([('poly', PolynomialFeatures(degree=11)),
                         ('linear', LinearRegression(fit_intercept=False))])
model_poly11.fit(X, data.Y)
data.plot(style='+')
plt.plot(xx, model_poly11.predict(xx))
plt.show()

In [ ]:
model_poly11['linear'].coef_

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
model = Pipeline([('poly', PolynomialFeatures(degree=11)),
                  ('linear', ElasticNet())])
model.fit(X, data.Y)
data.plot(style='+')
plt.plot(xx, model.predict(xx))
plt.show()


In [ ]:
model['linear'].coef_

In [ ]:
score = pd.DataFrame(columns=['mean', 'std'])
for n in range(10):
  model = Pipeline([('poly', PolynomialFeatures(degree=n)),
                    ('linear', ElasticNet(fit_intercept=False, tol=1e-4, random_state=13))])
  sc = cross_val_score(model, X, data.Y, cv=cv, scoring='neg_mean_squared_error')
  score.loc[n] = [-sc.mean(), sc.std()]

In [ ]:
score.plot(logy=True)
plt.show()

In [ ]:
score[1:8]['mean'].plot()
score['mean'].min()

In [ ]:
x_ = np.linspace(2, 16).reshape(-1, 1)

pd.DataFrame(
  {n: Pipeline([('poly', PolynomialFeatures(degree=n)),
                 ('linear', ElasticNet(fit_intercept=False))]).fit(X, data.Y).predict(x_)
                 for n in (2, 6)},
  index = x_[:,0]
).plot()
plt.plot(X, data.Y, '+')
plt.show()

## Логистическая 

Вместо вещественных чисел будем предсказывать числа из [0, 1]

Получается что мы решаем другую задачу и пришли к теме следующего семинара (классификация).


<img src='https://miro.medium.com/max/640/0*gKOV65tvGfY8SMem.png' width=600/>

$\displaystyle\sigma(x) = \frac{1}{1 + e^{-x}}$

Задача теперь формулируется так:

**Предсказания:** $$
y_{pred}(x, w) = \frac{1}{1 + e^{-\langle x, w \rangle}}
$$

**Функция потерь (LogLoss):** $$
L(y_{pred}, y) = -y\, log\,y_{pred} - (1-y)\,log\,(1-y_{pred})
$$

In [ ]:
from sklearn.metrics import log_loss

yhat = [x*0.01 for x in range(0, 101)]
losses_0 = [log_loss([0], [x], labels=[0,1]) for x in yhat]
losses_1 = [log_loss([1], [x], labels=[0,1]) for x in yhat]
plt.plot(yhat, losses_0, label='y=0')
plt.plot(yhat, losses_1, label='y=1')
plt.ylim((0, 5))
plt.legend()
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
X, y = make_blobs(n_samples=1000, centers=[[-2,0.5],[2,-0.5]], cluster_std=1, random_state=13)

colors = ("red", "k")
colored_y = np.zeros(y.size, dtype=str)

for i, cl in enumerate([0,1]):
    colored_y[y == cl] = str(colors[i])
    
plt.figure(figsize=(15,10))
plt.scatter(X[:, 0], X[:, 1], c=colored_y, edgecolors='K', s=80)
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

sns.countplot(x=y)
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)
clf.fit(X, y)

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

plt.figure(figsize=(15,8))

eps = 0.1
xx, yy = np.meshgrid(np.linspace(np.min(X[:,0]) - eps, np.max(X[:,0]) + eps, 500),
                     np.linspace(np.min(X[:,1]) - eps, np.max(X[:,1]) + eps, 500))

Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

Z = Z.reshape(xx.shape)

cmap_light = ListedColormap(['#FFAAAA', 'grey'])
plt.pcolormesh(xx, yy, Z, cmap=cmap_light)
plt.scatter(X[:, 0], X[:, 1], c=colored_y, edgecolors='K')
plt.show()

# Задание

1. Попытаться применить линейную регрессию к какой-либо паре зависимой и независимой переменных в собственном наборе данных. Нарисовать график зависимости y от x с линией. Кроссвалидацией оценить ошибку (любая метрика ошибки на ваш выбор).
2. Теперь при той же зависимой переменной y взять в качестве независимых переменных все (числовые) переменные набора данных. График нарисовать уже не получится, но кроссвалидацией всё равно можно оценить ошибку с той же выборкой. Стала ли ошибка меньше за счет того, что у модели больше информации?



